In [1]:
import torch
import torch.nn.functional as F
from torch import nn

# 自定义层

## 不带参数的层

- 构造一个没有任何参数的自定义层
    - 建立`CenteredLayer`类要
        - 从其输入中减去均值
        - 要构建它只需继承基础层类并实现前向传播功能

In [2]:
class CenteredLayer(nn.Module):
    # 继承基础层nn.Module
    def __init__(self):
        super().__init__()

    def forward(self, X):
        #减去均值
        return X - X.mean()

In [3]:
layer = CenteredLayer()  #实例化
layer(torch.FloatTensor([1, 2, 3, 4, 5])) # 各个输入值减去均值

tensor([-2., -1.,  0.,  1.,  2.])

- 将层作为组件合并到更复杂的模型中


In [4]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [6]:
Y = net(torch.rand(4, 8)) #检验模型运算是否正确。向该网络发送随机数据，检查均值是否为0
f'Y的均值为{Y.mean()}'

'Y的均值为9.313225746154785e-10'

- ```python
torch.rand(*size)
```
    - 生成从均匀分布$U(0,1)$中采样的随机张量
    - `size`：整数序列，定义生成的随机张量的形状

- ```python
torch.randn(*size)
```
    - 生成从正态分布$N(0,1)$中采样的随机张量

## 带参数的层

- 定义具有参数的层，这些参数可以通过训练进行调整

- 可以使用内置函数来创建参数，这些函数提供一些基本的管理功能,比如管理访问、初始化、共享、保存和加载模型参数
- 这样做的好处之一是：不需要为每个自定义层编写自定义的序列化程序

In [7]:
# 实现自定义版本的全连接层
# 该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数

class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 生成权重参数
        self.bias = nn.Parameter(torch.randn(units,))            # 生成偏置参数
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

- ```python
torch.nn.parameter.Parameter(data=None, requires_grad=True)
```
    - `data`：输入的张量
    - 生成张量，并且作为块Module的参数，出现在`net.parameters()`迭代器中

In [8]:
# 实例化MyLinear类并访问其模型参数

linear = MyLinear(5, 3)

print(f'linear的权重为\n{linear.weight}')

linear的权重为
Parameter containing:
tensor([[ 0.3162,  0.2361,  1.2055],
        [ 1.8310, -1.4149,  0.1414],
        [-0.6296, -0.0621, -0.3923],
        [-0.3969,  2.2812, -0.1973],
        [-1.7218,  2.4176, -3.2143]], requires_grad=True)


In [9]:
# 使用自定义层直接执行前向传播计算

linear(torch.rand(2, 5))  # 注意输入张量的列数与linear的参数的行数保持一致

tensor([[0.0000, 2.8299, 0.4563],
        [0.0000, 4.3097, 0.0000]])

In [10]:
# 使用自定义层构建模型

net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))  # 注意两个块参数形状的匹配

net(torch.rand(2, 64))

tensor([[0.],
        [0.]])